In [1]:
import pandas as pd

# ✅ Set your file path here
file_path = "/Users/raulgarcia/Downloads/Real_Estate_Sales_2001-2022_GL.csv"  # or provide full path like "C:/Users/Raul/Documents/..."

# ✅ Load the dataset
df = pd.read_csv(file_path)

# ✅ Preview the data
df.head()


/var/folders/7v/vvh437vj6hs1k4rlwqh7zzvc0000gn/T/ipykernel_1589/1618324201.py:7: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN
4,210288,2021,06/20/2022,Avon,12 BYRON DRIVE,179990.0,362500.0,0.4965,Residential,Condo,NaN,NaN,NaN,POINT (-72.879115982 41.773452988)


In [2]:
df.columns.tolist()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097629 entries, 0 to 1097628
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1097629 non-null  int64  
 1   List Year         1097629 non-null  int64  
 2   Date Recorded     1097627 non-null  object 
 3   Town              1097629 non-null  object 
 4   Address           1097578 non-null  object 
 5   Assessed Value    1097629 non-null  float64
 6   Sale Amount       1097629 non-null  float64
 7   Sales Ratio       1097629 non-null  float64
 8   Property Type     715183 non-null   object 
 9   Residential Type  699240 non-null   object 
 10  Non Use Code      313451 non-null   object 
 11  Assessor Remarks  171228 non-null   object 
 12  OPM remarks       13031 non-null    object 
 13  Location          298111 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 117.2+ MB


In [3]:
# Estimate Net Operating Income (NOI)
df["Estimated NOI"] = df["Sale Amount"] * 0.07


In [4]:
#simulated a stat for Square feet
import numpy as np

df["SqFt"] = np.where(
    df["Property Type"] == "Commercial",
    np.random.randint(3000, 15000, size=len(df)),
    np.random.randint(800, 3000, size=len(df))
)


In [16]:
np.random.seed(42)  # for reproducibility
df["Cap Rate"] = 0.05 + np.random.normal(loc=0.02, scale=0.01, size=len(df))
df["Price per SqFt"] = df["Sale Amount"] / df["SqFt"]


In [17]:
#Cleaning by eliminating any rows that have missing values 
df_clean = df[[
    "Town", "List Year", "Sale Amount", "Assessed Value",
    "Sales Ratio", "Property Type", "Estimated NOI",
    "Cap Rate", "SqFt", "Price per SqFt"
]].dropna(subset=["Town", "Sale Amount"])


In [18]:
#Download clean CSV
df_clean.to_csv("real_estate_clean.csv", index=False)


In [19]:
#SQLite 
import sqlite3

# Create an in-memory SQLite database (temporary, fast)
conn = sqlite3.connect(":memory:")

# Load cleaned DataFrame into SQL table
df_clean.to_sql("real_estate_sales", conn, index=False, if_exists="replace")


1097629

In [20]:
#Test SQL Connection
query = "SELECT * FROM real_estate_sales LIMIT 5;"
pd.read_sql_query(query, conn)


,Town,List Year,Sale Amount,Assessed Value,Sales Ratio,Property Type,Estimated NOI,Cap Rate,SqFt,Price per SqFt
0,Ansonia,2020,248400.0,133000.0,0.5354,Residential,17388.0,0.074967,2483,100.040274
1,Ansonia,2020,239900.0,110500.0,0.4606,Residential,16793.0,0.068617,1595,150.407524
2,Ansonia,2020,325000.0,150500.0,0.4630,Commercial,22750.0,0.076477,12230,26.573998
3,Ansonia,2020,202500.0,127400.0,0.6291,Residential,14175.0,0.085230,956,211.820084
4,Avon,2021,362500.0,179990.0,0.4965,Residential,25375.0,0.067658,2281,158.921526


In [21]:
#Top 10 Towns by Total Sale Volume
query = """
SELECT Town, SUM("Sale Amount") AS total_sales
FROM real_estate_sales
GROUP BY Town
ORDER BY total_sales DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)


,Town,total_sales
0,Greenwich,3.866514e+10
1,Stamford,3.309249e+10
2,Norwalk,1.677264e+10
3,Westport,1.533677e+10
4,Fairfield,1.325577e+10
5,Darien,1.167042e+10
6,New Canaan,1.094189e+10
7,Danbury,8.870835e+09
8,West Hartford,8.265551e+09
9,Bridgeport,8.028408e+09


In [24]:
# AVG Cap Rate by Property Type
query = """
SELECT "Property Type", ROUND(AVG("Cap Rate"), 4) AS avg_cap_rate
FROM real_estate_sales
WHERE "Cap Rate" IS NOT NULL
GROUP BY "Property Type"
ORDER BY avg_cap_rate DESC;
"""

pd.read_sql_query(query, conn)
#Higher cap rate is higher ROI but more risky
#Lower cap rate is more premium properties

,Property Type,avg_cap_rate
0,Commercial,0.0702
1,Two Family,0.0700
2,Single Family,0.0700
3,Residential,0.0700
4,None,0.0700
5,Vacant Land,0.0699
6,Three Family,0.0699
7,Industrial,0.0699
8,Four Family,0.0699
9,Condo,0.0699


In [25]:
#AVG Price per SqFT by Town
query = """
SELECT Town, ROUND(AVG("Price per SqFt"), 2) AS avg_price_per_sqft
FROM real_estate_sales
WHERE "Price per SqFt" IS NOT NULL
GROUP BY Town
ORDER BY avg_price_per_sqft DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)
#Greenwich had a lot of volume of purchase and high price per SqFt

,Town,avg_price_per_sqft
0,Willington,2093.35
1,Greenwich,1254.80
2,Darien,946.05
3,New Canaan,932.61
4,Westport,881.33
5,Weston,581.20
6,Wilton,542.53
7,Washington,531.03
8,Stamford,528.70
9,Ridgefield,447.68
